# 10 - FEMA Housing Registrants (IA)

Paged subset pull for TX/LA/MS/AL/FL; light clean with tract_geoid.


In [ ]:
import os, time, requests, pandas as pd
from pathlib import Path

PROJECT_ROOT = Path('/Users/liamguest/LProjects/AURA/AURA')
RAW = PROJECT_ROOT / 'data' / 'raw'
INT = PROJECT_ROOT / 'data' / 'interim'
RAW.mkdir(parents=True, exist_ok=True); INT.mkdir(parents=True, exist_ok=True)

STATE_ABBR = ['TX','LA','MS','AL','FL']
states_str = "','".join(STATE_ABBR)
OPENFEMA_HOUSING_V1 = 'https://www.fema.gov/api/open/v1/IndividualAssistanceHousingRegistrantsLargeDisasters'

select_cols = ','.join([
    'id','disasterNumber','damagedStateAbbreviation','damagedZipCode','ownRent','residenceType',
    'grossIncome','specialNeeds','tsaEligible','repairAssistanceEligible','replacementAssistanceEligible',
    'personalPropertyEligible','ppfvl','censusBlockId','censusYear'
])

top = 5000
max_pages = 5
skip = 0
frames = []
for page in range(max_pages):
    params = {
        '$filter': f"damagedStateAbbreviation in ('{states_str}')",
        '$select': select_cols,
        '$format': 'json',
        '$top': top,
        '$skip': skip,
    }
    r = requests.get(OPENFEMA_HOUSING_V1, params=params, timeout=60)
    r.raise_for_status()
    items = r.json().get('IndividualAssistanceHousingRegistrantsLargeDisasters', [])
    if not items:
        break
    frames.append(pd.DataFrame(items))
    skip += top
    time.sleep(0.2)

fema = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
raw_out = RAW / 'fema_housing_subset.json'
fema.to_json(raw_out, orient='records')
print('FEMA rows:', len(fema), '->', raw_out)

def to_tract_geoid(val):
    s = str(val) if pd.notna(val) else ''
    return s[:11] if len(s) >= 11 else None

if not fema.empty:
    fema['tract_geoid'] = fema['censusBlockId'].map(to_tract_geoid)
    clean_out = INT / 'fema_housing_subset_clean.csv'
    fema.to_csv(clean_out, index=False)
    print('Wrote clean subset:', clean_out, 'rows:', len(fema))
